In [1]:
import torch 
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# %cd ../

In [2]:
import sys 
sys.path.append('/workspace/Walk_Video_PyTorch/project')
sys.path

['/workspace/Walk_Video_PyTorch/project/tests',
 '',
 '/usr/lib/python3.9/site-packages',
 '/usr/lib/python39.zip',
 '/usr/lib/python3.9',
 '/usr/lib/python3.9/lib-dynload',
 '/usr/local/lib/python3.9/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/workspace/Walk_Video_PyTorch/project']

In [3]:
from models.pytorchvideo_models import WalkVideoClassificationLightningModule
from pytorch_lightning import Trainer

from IPython.display import clear_output

clear_output()

In [4]:
from pytorch_lightning import seed_everything

seed_everything(42, workers=True)

Global seed set to 42


42

In [5]:
from main import get_parameters

opt, unknown = get_parameters()
opt.num_workers = 8
opt.batch_size = 8
opt.gpu_num = 1
opt.model = "resnet"
opt.model_depth = 101
opt.clip_duarion = 1
opt.version = '0809' + '_' + opt.model + '_depth' + str(opt.model_depth)


In [6]:
from utils.utils import get_ckpt_path

model = WalkVideoClassificationLightningModule(opt)

# get last ckpt path
ckpt_path = get_ckpt_path(opt)


model = model.load_from_checkpoint(ckpt_path)

model.eval()

clear_output()
print(ckpt_path)


/workspace/Walk_Video_PyTorch/logs/resnet/0809_resnet_depth101/checkpoints/epoch=78-step=16432.ckpt


In [12]:
from dataloader.data_loader import WalkDataModule
from pytorch_lightning import loggers as pl_loggers 

# load test dataset 
module = WalkDataModule(opt)
module.setup()
test_data = module.test_dataloader()

# for the tensorboard
tb_logger = pl_loggers.TensorBoardLogger(save_dir="/workspace/Walk_Video_PyTorch/project/tests/logs", name=opt.model, version=opt.version)

trainer = Trainer(
    accelerator="auto",
    devices=1,
    gpus=opt.gpu_num,
    logger=tb_logger,
    max_epochs=1,
    deterministic=True,
)

trainer.test(dataloaders=module, model=model)


/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:460: UserWarning: The flag `devices=1` will be ignored, instead the device specific number 1 will be used
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/trainer.py:726: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
# trainer.validate(model=model, dataloaders=module)

In [ ]:
input_data = next(iter(test_data))


In [ ]:
input_data['video'].shape

input_data["video_name"]

# input_data['video'][0][0][0][0]

['20200528_DHS_lat_V1-0030.mp4',
 '20200528_DHS_lat_V1-0030.mp4',
 '20200528_DHS_lat_V1-0030.mp4',
 '20200528_DHS_lat_V1-0030.mp4',
 '20200528_DHS_lat_V1-0030.mp4',
 '20200528_DHS_lat_V1-0030.mp4',
 '20200528_DHS_lat_V1-0030.mp4',
 '20200528_DHS_lat_V1-0030.mp4']

In [ ]:
input_data['label']

tensor([1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
preds = model(input_data['video'])

In [ ]:
post_act = torch.nn.Softmax(dim=1)
preds = post_act(preds)
pred_classes = preds.topk(k=1).indices

In [ ]:
pred_classes

tensor([[0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0]])

In [ ]:
from torch import softmax


pred_chagne = softmax(preds, dim=-1)
pred_chagne

tensor([[0.7307, 0.2693],
        [0.7309, 0.2691],
        [0.4216, 0.5784],
        [0.7273, 0.2727],
        [0.7244, 0.2756],
        [0.7303, 0.2697],
        [0.7310, 0.2690],
        [0.7311, 0.2689]], grad_fn=<SoftmaxBackward0>)

In [ ]:
from utils.metrics import get_Accuracy

accuracy = get_Accuracy()

accuracy(pred_chagne, input_data['label'])

tensor(0.1250)

In [ ]:
classname = {}

classname[0] = 'asd'
classname[1] = 'asd_not'

In [ ]:
real_calss = []

for i in input_data['label'].tolist():
    real_calss.append(classname[i])

In [ ]:
real_calss

['asd_not',
 'asd_not',
 'asd_not',
 'asd_not',
 'asd_not',
 'asd_not',
 'asd_not',
 'asd_not']

In [ ]:
# pred_class_names = []
# for num in range(opt._BATCH_SIZE):
#     for i in pred_classes[i]:
#         pred_class_names.append(classname[int(i)])


pred_class_names = [classname[int(i)] for i in pred_classes]
print("Predicted labels: %s" % ", ".join(pred_class_names))
print("real label: %s" % ",".join(real_calss))

Predicted labels: asd, asd, asd_not, asd, asd, asd, asd, asd
real label: asd_not,asd_not,asd_not,asd_not,asd_not,asd_not,asd_not,asd_not


In [ ]:
pred_class_names == real_calss

False

In [ ]:
result = []

for i in range(len(real_calss)):
    if pred_class_names[i] == real_calss[i]:
        result.append("true")
    else:
        result.append("false")

result

['false', 'false', 'true', 'false', 'false', 'false', 'false', 'false']